In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Swarnendu\\Desktop\\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Swarnendu\\Desktop\\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
import sys
sys.path.append('C:/Users/Swarnendu/Desktop/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC/src')


In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
from pathlib import Path
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier.entity.config_entity import DataIngestionConfig

# Utility function to create directories
def create_directories(paths):
    for path in paths:
        if not path.exists():
            path.mkdir(parents=True, exist_ok=True)

class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = Path(r"C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml"),
        params_filepath: Path = Path(r"C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\params.yaml")
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Create the artifacts and data_ingestion directories if they do not exist
        self.artifacts_dir = Path("C:/Users/Swarnendu/Desktop/End-to-end-Chest-Cancer-Classification-using-MLflow-DVC/artifacts")
        self.data_ingestion_dir = self.artifacts_dir / "data_ingestion"
        create_directories([self.artifacts_dir, self.data_ingestion_dir])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # Ensure that the root directory for data ingestion exists
        create_directories([self.data_ingestion_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=self.data_ingestion_dir,
            source_URL="https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing",
            local_data_file=self.data_ingestion_dir / "data.zip",  # Correct file path for ZIP file
            unzip_dir=self.data_ingestion_dir
        )

        return data_ingestion_config

# Example usage



In [9]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [10]:

from cnnClassifier.entity.config_entity import DataIngestionConfig

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Fetch data from the URL and save it locally.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = str(self.config.local_data_file)  # Convert Path to string
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)  # Pass string path

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")
            return zip_download_dir
        except Exception as e:
            logger.error(f"Failed to download file: {e}")
            raise e

    def extract_zip_file(self):
        """
        Extract the zip file into the data directory.
        """
        try:
            unzip_path = str(self.config.unzip_dir)  # Convert Path to string
            os.makedirs(unzip_path, exist_ok=True)
            with zipfile.ZipFile(str(self.config.local_data_file), 'r') as zip_ref:  # Convert Path to string
                zip_ref.extractall(unzip_path)
            logger.info(f"Files extracted successfully to {unzip_path}")
        except FileNotFoundError as e:
            logger.error(f"Error: {e}. Please check if the zip file exists at the specified path.")
        except Exception as e:
            logger.error(f"An unexpected error occurred: {e}")
            raise e


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-08-17 12:05:45,020: INFO: common: yaml file: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\config\config.yaml loaded successfully]
[2024-08-17 12:05:45,028: INFO: common: yaml file: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\params.yaml loaded successfully]
[2024-08-17 12:05:45,032: INFO: 3241792270: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\artifacts\data_ingestion\data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=b0b3b657-b735-4a29-a188-2bbec330aa62
To: C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 49.0M/49.0M [00:15<00:00, 3.19MB/s]

[2024-08-17 12:06:06,063: INFO: 3241792270: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\artifacts\data_ingestion\data.zip]


[2024-08-17 12:06:06,948: INFO: 3241792270: Files extracted successfully to C:\Users\Swarnendu\Desktop\End-to-end-Chest-Cancer-Classification-using-MLflow-DVC\artifacts\data_ingestion]
